In [1]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup


link = 'https://github.com/RahulWadwani'

soup = BeautifulSoup(requests.get(link).content,'html.parser')

In [2]:
def social_account(urls):
    accounts = []
    for url in urls:
        if 'kaggle' in url:
            accounts.append('Kaggle')
        elif 'instagram' in url:
            accounts.append('Instagram')
        elif 'facebook' in url:
            accounts.append('Facebook')
        elif 'linkedin' in url:
            accounts.append('Linkedin')
        elif 'github' in url:
            accounts.append('Github')
        else:
            accounts.append('Personal website')
    return ','.join(accounts)

In [3]:
user_name  = soup.find('span',class_ = 'p-name vcard-fullname d-block overflow-hidden').text.strip()
user_id    = soup.find('span',class_ = 'p-nickname vcard-username d-block').text.strip()
bio        = soup.find('div',class_ = 'js-profile-editable-area d-flex flex-column d-md-block').find('div').text

data = []
for i in soup.find_all('a',class_ = 'Link--secondary no-underline no-wrap'):
    if 'follower' in (i.text):
        follower = int(i.find('span').text.strip())
    elif 'following' in i.text:
        following = int(i.find('span').text.strip())
    try:        
        content  = [int(content.find('span').text.strip()) for content in soup.find('nav',class_ ='UnderlineNav-body width-full p-responsive js-sidenav-container-pjax').find_all('a')[1:]]
        repos    = content[0]
        project  = content[1]
        packages = content[2]
        stars    = content[3]
    except:
        repos    = np.nan
        project  = np.nan
        packages = np.nan
        stars    = np.nan
    try:
        url = [links.find('a')['href'] for links in soup.find_all('li',class_='vcard-detail pt-1')]
        account = social_account(url)
    except:
        url = np.nan
        account = 'no social account has been linked'


data.append([user_id , user_name , bio , follower , following ,url,account, repos , project , packages , stars])

df = pd.DataFrame(data,columns = ['user_id' , 'user_name' , 'bio' , 'follower' , 'following' ,'url','account', 'repos' , 'project' , 'packages' , 'stars'])
df.head()




,user_id,user_name,bio,follower,following,url,account,repos,project,packages,stars
0,RahulWadwani,Rahul,I am Rahul Wadwani. A Student at Mumbai Univer...,1,4,[https://www.kaggle.com/rahulwadwani],Kaggle,6,0,0,9


In [4]:
# scraping the repos

In [5]:
link = 'https://github.com/RahulWadwani?tab=repositories'
soup = BeautifulSoup(requests.get(link).content,'html.parser')
print(soup)


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-719f1193e0c0.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-0c343b529849.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/assets/dark_dimmed-f22da508b62a.c